In [1]:
import os
import pandas as pd
from frame import Frame
from testing import z_score_trading
from google.cloud import storage
from google.cloud import bigquery
from data_query import fetch_NASDAQ100_index
from data_query import fetch_NASDAQ100_all_components
from data_query import fetch_SP500_index
from data_query import fetch_SP500_all_components
from data_query import fetch_ftse100_index
from data_query import fetch_ftse100_all_components
from PCA_function import rolling_pca_weights
from preprocessing import preprocessing_X
from sklearn.decomposition import PCA
#
#-----Pulling data from Big Query
#
index_selected='ftse'
#
if index_selected=='nasdaq':
    target_df= fetch_NASDAQ100_index()
    underlying_df=fetch_NASDAQ100_all_components()

if index_selected=='sp500':
    target_df= fetch_SP500_index()
    underlying_df=fetch_SP500_all_components()

if index_selected=='ftse':
    target_df= fetch_ftse100_index()
    underlying_df=fetch_ftse100_all_components()
#
#-----pre-processing the components
processed_df=preprocessing_X(underlying_df)
#
#-----PCA function
#
# Define input variables
X_log=processed_df
n_stocks = 30               # number of stocks used for the replication
window = 30                 # period the trading strat goes
n_pcs = 3                   # number of eigenvectors
#
# Get weights
rep_pf = rolling_pca_weights(X_log, n_stocks, window, n_pcs)


pca_weights_df=rep_pf
#underlying_df=stock_price
#target_df=target_close_price
cal_days=60                 # number of days for the z score
trade_days=30               # maximum number of trading days
thresholds=[2,200,-2,-200]  # thresholds for trading signals
                            # [short minimum threshold, short maximum threshold, long minimum threshold, long maximum threshold]
exit_levels=[0.5,-0.5]      # thresholds for closing a trade
                            # [exit level long position, exit level short position]

In [3]:
bt_result=z_score_trading(pca_weights_df, underlying_df, target_df, cal_days, trade_days, thresholds, exit_levels, True)

      z_score  direction  target entry  replication entry     Day 1     Day 2  \
61  -1.427260        0.0   7273.399902        7201.110832 -1.534153 -1.674400   
62  -0.657440        0.0   7045.000000        7014.594191 -0.816673  0.839912   
63  -0.463811        0.0   7016.299805        7071.825560  1.271381  0.474550   
64  -0.452611        0.0   7121.799805        7129.022683  0.215063  0.553077   
65   0.490890        0.0   7152.100098        7154.740237  0.076594  0.716489   
..        ...        ...           ...                ...       ...       ...   
750 -2.890969        1.0   8871.299805        9008.853905  3.422073 -3.530187   
751  4.172047       -1.0   8759.000000        8654.369886 -2.826709 -0.910816   
752  1.044046        0.0   8755.799805        8713.929481 -1.333857 -0.943933   
753 -2.255070        1.0   8682.799805        8666.375804 -0.482005 -0.250266   
754  2.655596       -1.0   8679.900391        8679.900391  0.012865       NaN   

        Day 3     Day 4    

In [6]:
bt_result.columns

Index(['z_score', 'direction', 'target entry', 'replication entry', 'Day 1',
       'Day 2', 'Day 3', 'Day 4', 'Day 5', 'Day 6', 'Day 7', 'Day 8', 'Day 9',
       'Day 10', 'Day 11', 'Day 12', 'Day 13', 'Day 14', 'Day 15', 'Day 16',
       'Day 17', 'Day 18', 'Day 19', 'Day 20', 'Day 21', 'Day 22', 'Day 23',
       'Day 24', 'Day 25', 'Day 26', 'Day 27', 'Day 28', 'Day 29',
       'replication exit', 'target exit', 'target return',
       'replication return'],
      dtype='object')

In [8]:
cum_return=bt_result['direction'] * (bt_result['target return']-bt_result['replication return'])

In [10]:
cum_return.sum()

np.float64(-0.30288746263287347)

In [12]:
bt_return=bt_result['direction'] * (bt_result['target return']-bt_result['replication return'])

In [13]:
bt_return

61    -0.000000
62    -0.000000
63    -0.000000
64     0.000000
65    -0.000000
         ...   
750   -0.006462
751    0.012017
752         NaN
753         NaN
754         NaN
Length: 694, dtype: float64

In [19]:
bt_number_of_trades= np.count_nonzero(bt_result['direction'], axis=0)

In [20]:
bt_number_of_trades

np.int64(52)

In [21]:
 capital=pd.DataFrame(0, index=np.arange(len(bt_return),columns='Capital')

    for i in range(1,len(bt_return),1):
        capital.iloc[i]=capital.iloc[i-1]*exp(bt_return[i])


SyntaxError: '(' was never closed (1811165680.py, line 1)

In [54]:
import math
bt_to_API=pd.DataFrame(bt_result['direction'] * (bt_result['target return']-bt_result['replication return']))
bt_to_API.columns=['log return by trade']
bt_to_API['direction']=bt_result['direction']


In [60]:
bt_to_API

,log return by trade,direction,capital
61,-0.000000,0.0,100
62,-0.000000,0.0,100
63,-0.000000,0.0,100
64,0.000000,0.0,100
65,-0.000000,0.0,100
...,...,...,...
750,-0.006462,1.0,100
751,0.012017,-1.0,100
752,NaN,0.0,100
753,NaN,1.0,100


In [59]:
bt_to_API['capital']=100

In [66]:
bt_to_API.loc[150,'log return by trade']

np.float64(-0.0)

In [67]:
for i, row in bt_to_API.iterrows():
    bt_to_API.loc[i+1,'capital']=bt_to_API.loc[i,'capital']*math.exp(bt_to_API.loc[i,'log return by trade'])

/tmp/ipykernel_43300/4161105311.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '99.42544331073904' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  bt_to_API.loc[i+1,'capital']=bt_to_API.loc[i,'capital']*math.exp(bt_to_API.loc[i,'log return by trade'])


In [68]:
bt_to_API

,log return by trade,direction,capital
61,-0.000000,0.0,100.0
62,-0.000000,0.0,100.0
63,-0.000000,0.0,100.0
64,0.000000,0.0,100.0
65,-0.000000,0.0,100.0
...,...,...,...
751,0.012017,-1.0,NaN
752,NaN,0.0,NaN
753,NaN,1.0,NaN
754,NaN,-1.0,NaN


In [70]:
bt_to_API.head(50)

,log return by trade,direction,capital
61,-0.000000,0.0,100.000000
62,-0.000000,0.0,100.000000
63,-0.000000,0.0,100.000000
64,0.000000,0.0,100.000000
65,-0.000000,0.0,100.000000
66,0.000000,0.0,100.000000
67,-0.000000,0.0,100.000000
68,0.000000,0.0,100.000000
69,-0.000000,0.0,100.000000
70,0.000000,0.0,100.000000
